  ![ImdbIcon](images/pets.jpg)
# Petfinder: Predicting Adoption Speed 

In [1]:
import petpy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import text

from nltk.sentiment.vader import SentimentIntensityAnalyzer

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import scipy.stats as stats
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

In [2]:
pd.set_option('display.max_columns', 500)

In [32]:
# Using petpy wrapper with petfinder API keys
pf = petpy.Petfinder(key='mVBo96VLQBNZjBQJrO37FI52Rh32Hzl7x01Ie9VOGsuTBHdlga', secret='voFRNPPlmWoCIz8BtGaiLFlJgD58j6NaOu3kVqYp')

In [33]:
# Requesting my data from 2019 through 2020
last2 = pf.animals(animal_type='dog', status='adopted', results_per_page=50, pages=200, location=94602, before_date = '2019-01-26 18:03:11', after_date='2018-12-31', return_df=True)

Rate Limit Exceeded


/Users/jbow/opt/anaconda3/lib/python3.8/site-packages/petpy/api.py:1049: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  results_df = json_normalize(results[key])


In [36]:
#last.shape

(850, 50)

In [5]:
#dogs1.drop(columns=['primary_photo_cropped'], inplace=True)

In [6]:
#statu5 = pd.read_csv('./petfinder_data/statu5.csv')

In [7]:
#dogs = pd.read_csv('./petfinder_data/dogsca.csv')

In [8]:
# Had to make multiple requests, so concatenating them
dogs1 = pd.concat((last, last_1))

In [9]:
#dogs1.sort_values(by=['published_at'], inplace=True)

In [10]:
#dogs1.drop_duplicates(subset=['id'], inplace=True)

In [11]:
#dogs1.columns.values[1] = "org_id"

In [12]:
#dogs1.to_csv('./petfinder_data/dogs.csv', index = False)

---------

## Imports and Data Cleaning

id: Unique pet ID \
organization_id: ID of shelter \
url: unique petfinder post url \
age: puppy(0-6 months), young(6-24 months), Adult(2-8 years), Senior(>8 years) \
gender: male(), female() \
size: small(0-25lbs), medium(26-60lbs), large(61-100lbs), extra large(>101lbs) \
coat: short, medium, long, wire, curly, hairless \
tags: key word descriptors for animal \
name \
description: unique description of dog, sometimes contains background, health, etc. \
photos: \
videos: has video (n:0, y:1) \
breeds.primary: primary dog breed \ 
breeds.secondary: secondary breed if applicable \
breeds.mixed: if a dog is purebred
colors.primary: black, white/cream, brown/chocolate, bicolor, apricot/beige, tricolor(brown, black, white), yellow/tan/blond/fawn, gray/blue/silver, brindle, red/chestnut/orange, golden, sable, merle(blue), merle(red), harlequin \
colors.secondary: coat color, dog is seen to have secondary (usually less dominant) color \
colors.tertiary: coat color, if dog is seen to have third (usually least dominant) color \
fixed: spayed/neutered n:0, y:1 \
house_trained: n:0, y:1 \
special_needs: n:0, y:1 \
shots_current: n:0, y:1 \
good_with_kids: n:0, y:1 \
good_with_dogs: n:0, y:1 \
good_with_cats: n:0, y:1 \
primary_photo_cropped.small:
primary_photo_cropped.medium:
primary_photo_cropped.large:
primary_photo_cropped.full: \
city: \
days_on_petfinder: 


In [13]:
#statu5 = pd.read_csv('./petfinder_data/statu5.csv')

In [3]:
dogs1 = pd.read_csv('./petfinder_data/dogs.csv')

In [5]:
dogs1.videos.value_counts()

1    20020
Name: videos, dtype: int64

In [15]:
#dogs.to_csv('./petfinder_data/dogsca.csv', index = False)

In [39]:
dogs1.drop_duplicates(subset=['id'], inplace=True)

In [40]:
dogs1.sort_values(by=['published_at'], inplace=True)

In [41]:
dogs1['published_at']= pd.to_datetime(dogs1['published_at'])
dogs1['status_changed_at']= pd.to_datetime(dogs1['status_changed_at'])

In [42]:
dogs1.status.value_counts()

adopted    20020
Name: status, dtype: int64

In [20]:
#dogs.isnull().sum()

In [43]:
dogs1.rename(columns={"attributes.spayed_neutered": "fixed", "attributes.house_trained": "house_trained", 'attributes.special_needs': 'special_needs', 
                      'attributes.shots_current': 'shots_current','environment.children': 'good_with_kids', 'environment.dogs': 'good_with_dogs', 
                      'environment.cats': 'good_with_cats', 'contact.address.city': 'city', 'contact.address.postcode': 'zipcode' }, inplace=True)
dogs1.head(1)

,id,organization_id,url,type,species,age,gender,size,coat,tags,name,description,organization_animal_id,photos,videos,status,status_changed_at,published_at,distance,breeds.primary,breeds.secondary,breeds.mixed,colors.primary,colors.secondary,colors.tertiary,fixed,house_trained,attributes.declawed,special_needs,shots_current,good_with_kids,good_with_dogs,good_with_cats,primary_photo_cropped.small,primary_photo_cropped.medium,primary_photo_cropped.large,primary_photo_cropped.full,contact.email,contact.phone,contact.address.address1,contact.address.address2,city,contact.address.state,zipcode,contact.address.country,animal_id,animal_type,organization_id.1,has_photo,color_Apricot / Beige,color_Bicolor,color_Black,color_Brindle,color_Brown / Chocolate,color_Golden,color_Gray / Blue / Silver,color_Harlequin,color_Merle (Blue),color_Merle (Red),color_Red / Chestnut / Orange,color_Sable,"color_Tricolor (Brown, Black, & White)",color_White / Cream,color_Yellow / Tan / Blond / Fawn,breeds_Affenpinscher,breeds_Afghan Hound,breeds_Airedale Terrier,breeds_Akbash,breeds_Akita,breeds_Alaskan Malamute,breeds_American Bulldog,breeds_American Bully,breeds_American Eskimo Dog,breeds_American Foxhound,breeds_American Hairless Terrier,breeds_American Staffordshire Terrier,breeds_Anatolian Shepherd,breeds_Aussiedoodle,breeds_Australian Cattle Dog / Blue Heeler,breeds_Australian Kelpie,breeds_Australian Shepherd,breeds_Australian Terrier,breeds_Basenji,breeds_Basset Hound,breeds_Beagle,breeds_Bearded Collie,breeds_Beauceron,breeds_Bedlington Terrier,breeds_Belgian Shepherd / Malinois,breeds_Belgian Shepherd / Sheepdog,breeds_Belgian Shepherd / Tervuren,breeds_Bernese Mountain Dog,breeds_Bichon Frise,breeds_Black Labrador Retriever,breeds_Black Mouth Cur,breeds_Black Russian Terrier,breeds_Black and Tan Coonhound,breeds_Bloodhound,breeds_Blue Lacy,breeds_Bluetick Coonhound,breeds_Boerboel,breeds_Bolognese,breeds_Border Collie,breeds_Border Terrier,breeds_Boston Terrier,breeds_Bouvier des Flandres,breeds_Boxer,breeds_Briard,breeds_Brittany Spaniel,breeds_Brussels Griffon,breeds_Bull Terrier,breeds_Bullmastiff,breeds_Cairn Terrier,breeds_Canaan Dog,breeds_Cane Corso,breeds_Cardigan Welsh Corgi,breeds_Carolina Dog,breeds_Catahoula Leopard Dog,breeds_Cattle Dog,breeds_Cavalier King Charles Spaniel,breeds_Chesapeake Bay Retriever,breeds_Chihuahua,breeds_Chinese Crested Dog,breeds_Chinook,breeds_Chiweenie,breeds_Chocolate Labrador Retriever,breeds_Chow Chow,breeds_Clumber Spaniel,breeds_Cockapoo,breeds_Cocker Spaniel,breeds_Collie,breeds_Coonhound,breeds_Corgi,breeds_Coton de Tulear,breeds_Dachshund,breeds_Dalmatian,breeds_Doberman Pinscher,breeds_Dogo Argentino,breeds_Dogue de Bordeaux,breeds_Dutch Shepherd,breeds_English Bulldog,breeds_English Cocker Spaniel,breeds_English Foxhound,breeds_English Pointer,breeds_English Setter,breeds_English Shepherd,breeds_English Springer Spaniel,breeds_Eskimo Dog,breeds_False,breeds_Finnish Spitz,breeds_Flat-Coated Retriever,breeds_Fox Terrier,breeds_Foxhound,breeds_French Bulldog,breeds_German Shepherd Dog,breeds_German Shorthaired Pointer,breeds_German Spitz,breeds_German Wirehaired Pointer,breeds_Giant Schnauzer,breeds_Golden Retriever,breeds_Goldendoodle,breeds_Great Dane,breeds_Great Pyrenees,breeds_Greater Swiss Mountain Dog,breeds_Greyhound,breeds_Harrier,breeds_Havanese,breeds_Hound,breeds_Husky,breeds_Ibizan Hound,breeds_Irish Setter,breeds_Irish Terrier,breeds_Irish Wolfhound,breeds_Italian Greyhound,breeds_Jack Russell Terrier,breeds_Japanese Chin,breeds_Jindo,breeds_Keeshond,breeds_Kishu,breeds_Klee Kai,breeds_Labradoodle,breeds_Labrador Retriever,breeds_Lakeland Terrier,breeds_Lancashire Heeler,breeds_Leonberger,breeds_Lhasa Apso,breeds_Maltese,breeds_Maltipoo,breeds_Manchester Terrier,breeds_Maremma Sheepdog,breeds_Mastiff,breeds_McNab,breeds_Miniature Dachshund,breeds_Miniature Pinscher,breeds_Miniature Poodle,breeds_Miniature Schnauzer,breeds_Mixed Breed,breeds_Morkie,breeds_Mountain Cur,breeds_Mountain D

In [44]:
dogs1['city']=dogs1['city'].replace('modesto','Modesto')

## Feature Engineering

In [49]:
# Converting videos column to 1 if post has a video, 0 if not
dogs1['videos'] = np.where(dogs1['videos']!= '[]', 1, 0)

In [50]:
# Adding a column, 1 if post has photos, 0 if not
dogs1['has_photo'] = np.where(dogs1['photos']!= '[]', 1, 0)

In [51]:
# Dummifying categorical columns
dogs1 = pd.get_dummies(dogs1, columns=['age', 'gender', 'size', 'coat', 'city'])

KeyError: "None of [Index(['age', 'gender', 'size', 'coat', 'city'], dtype='object')] are in the [columns]"

In [53]:
dogs1.videos.value_counts()

1    20020
Name: videos, dtype: int64

In [26]:
dogs1['good_with_kids'] = dogs1['good_with_kids'].fillna(2)
dogs1['good_with_dogs'] = dogs1['good_with_dogs'].fillna(2)
dogs1['good_with_cats'] = dogs1['good_with_cats'].fillna(2)

dogs1['fixed'] = (dogs1['fixed'] * 1).astype(int)
dogs1['house_trained'] = (dogs1['house_trained'] * 1).astype(int)
dogs1['shots_current'] = (dogs1['shots_current'] * 1).astype(int)
dogs1['good_with_kids'] = (dogs1['good_with_kids'] * 1).astype(int)
dogs1['good_with_dogs'] = (dogs1['good_with_dogs'] * 1).astype(int)
dogs1['good_with_cats'] = (dogs1['good_with_cats'] * 1).astype(int)
dogs1['special_needs'] = (dogs1['special_needs'] * 1).astype(int)

In [27]:
def dummy(df, label):
    cols = [col for col in df if label in col]
    dummy_dict = {}
    main_df = pd.get_dummies(df[cols[0]],prefix = label)
    for col in cols[1:]:
        dummy_dict[col] = pd.get_dummies(df[col],prefix = label)
    for col in dummy_dict:
        main_df = main_df.add(dummy_dict[col], fill_value = 0).gt(0)*1
    return main_df

In [28]:
# Running function and merging dummified df back to original df

dummy_color_dog = dummy(dogs1, 'color') 
dogs1 = pd.merge(dogs1, dummy_color_dog, left_index=True, right_index=True)

In [29]:
dogs1.drop(columns=['breeds.unknown'], inplace=True) # All breeds are known or guessed at so dropping this column
dummy_color_dog = dummy(dogs1, 'breeds') 
dogs1 = pd.merge(dogs1, dummy_color_dog, left_index=True, right_index=True)

KeyError: "['breeds.unknown'] not found in axis"

In [ ]:
dogs1['days_on_petfinder'] = dogs1['status_changed_at'] - dogs1['published_at']
dogs1['days_on_petfinder'] = dogs1['days_on_petfinder'] / np.timedelta64(1, 'D')
dogs1['days_on_petfinder'] = dogs1['days_on_petfinder'].round()

In [ ]:
dogs1.drop_duplicates(subset=['id'], inplace=True)

In [ ]:
dogs1.drop(columns=['type', 'species', 'tags', 'organization_animal_id', 'status', 'breeds.primary', 'breeds.secondary', 'breeds.mixed', 
                   'colors.primary', 'colors.secondary', 'colors.tertiary', 'attributes.declawed', 'primary_photo_cropped.small', 'primary_photo_cropped.medium',
                   'primary_photo_cropped.large', 'primary_photo_cropped.full', 'contact.email', 'contact.phone', 'contact.address.address1',
                   'contact.address.address2', 'contact.address.country', 'state', 'animal_id', 'animal_type', 'organization_id', 'breeds_True'], inplace=True)

In [ ]:
dogs1.shape

In [ ]:
dogs1.to_csv('./petfinder_data/cleandogs.csv', index = False)

In [ ]:
dogs1.loc[dogs1['days_on_petfinder'] < 14, 'days_on_petfinder'] = 1

In [ ]:
dogs1.loc[dogs1['days_on_petfinder'] >= 14, 'days_on_petfinder'] = 0

In [ ]:
dogs1.days_on_petfinder.value_counts()

In [ ]:
#dogs1.to_csv('./petfinder_data/cleandogs.csv', index = False)

In [ ]:
dogs1.head()

In [ ]:
# Creating list of columns to drop
drops = ['id', 'org_id', 'url', 'name', 'photos', 'status_changed_at', 'published_at', 'distance','description', 'days_on_petfinder']

# Dropping columns from data frame and dummifying categorical columns
X = dogs1.drop(columns=drops)
y = dogs1['days_on_petfinder']

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.30,
    stratify=y,
    random_state=42)

In [ ]:
ss =StandardScaler()
Z_train = ss.fit_transform(X_train)
Z_test = ss.transform(X_test)

In [ ]:
%%time
lr = LogisticRegression(max_iter=5000)
lr.fit(X_train, y_train)
lr_train = lr.score(X_train, y_train)
lr_test = lr.score(X_test, y_test)

In [ ]:
print(f'train score: {lr_train}')
print(f'test score: {lr_test}')

In [ ]:
# Baseline
dogs['days_on_petfinder'].value_counts(normalize=True)

In [ ]:
y_pred=lr.predict(X_test)
y_pred[0:20]

In [ ]:
confusion = confusion_matrix(y_test, y_pred)
print('Confusion Matrix\n')
print(confusion)